# Step 4 - Analysis of bicycle network results
## Project: Growing Urban Bicycle Networks

This notebook takes the existing infrastructure, the results from 03_poi_based_generation and calculates/analyzes a number of measures:
* cost (length)
* coverage  
* directness  
* efficiency
* overlap with existing networks

Contact: Michael Szell (michael.szell@gmail.com)  
Created: 2020-07-08  
Last modified: 2024-09-24

## Preliminaries

### Parameters

In [ ]:
debug = False # If True, will produce plots and/or verbose output to double-check
rerun_existing = False # If True, will re-run the costly analysis of existing infra even if files already exist.
%run -i "../parameters/parameters.py"

### Setup

In [ ]:
%run -i path.py
%run -i setup.py
if not debug: # Only do this if sure the code is bug-free!
    warnings.filterwarnings('ignore')

%load_ext watermark
%watermark -n -v -m -g -iv

### Functions

In [ ]:
%run -i functions.py

## Analyze existing infrastructure

In [ ]:
for placeid, placeinfo in tqdm(cities.items(), desc = "Cities"): 
    # output_place is one static file for the existing city. This can be compared to the generated infrastructure.
    # Make a check if this file was already generated - it only needs to be done once. If not, generate it:
    filename = placeid + "_existing.csv"
    if rerun_existing or not os.path.isfile(PATH["results"] + placeid + "/" + filename):
        print(placeid + ": Analyzing existing infrastructure... (this can take several minutes)")
        empty_metrics = {
                         "length":0,
                         "length_lcc":0,
                         "coverage": 0,
                         "directness": 0,
                         "directness_lcc": 0,
                         "poi_coverage": 0,
                         "components": 0,
                         "efficiency_global": 0,
                         "efficiency_local": 0,
                         "efficiency_global_routed": 0,
                         "efficiency_local_routed": 0,
                         "directness_lcc_linkwise": 0,
                         "directness_all_linkwise": 0
                        }
        output_place = {}
        for networktype in networktypes:
            output_place[networktype] = copy.deepcopy(empty_metrics)

        # Analyze all networks     
        Gs = {}
        for networktype in networktypes:
            try:
                if networktype != "biketrack_onstreet" and networktype != "bikeable_offstreet":
                    Gs[networktype] = csv_to_ig(PATH["data"] + placeid + "/", placeid, networktype)
                    Gs[networktype + "_simplified"] = csv_to_ig(PATH["data"] + placeid + "/", placeid, networktype + "_simplified")
                elif networktype == "biketrack_onstreet":
                    Gs[networktype] = intersect_igraphs(Gs["biketrack"], Gs["carall"])
                    Gs[networktype + "_simplified"] = intersect_igraphs(Gs["biketrack_simplified"], Gs["carall_simplified"])
                elif networktype == "bikeable_offstreet":
                    G_temp = copy.deepcopy(Gs["bikeable"])
                    delete_overlaps(G_temp, Gs["carall"])
                    Gs[networktype] = G_temp
                    G_temp = copy.deepcopy(Gs["bikeable_simplified"])
                    delete_overlaps(G_temp, Gs["carall_simplified"])
                    Gs[networktype + "_simplified"] = G_temp
            except:
                pass
        
        with open(PATH["data"] + placeid + "/" + placeid + '_poi_' + poi_source + '_nnidscarall.csv') as f:
            nnids = [int(line.rstrip()) for line in f]

            
        covs = {}
        for networktype in tqdm(networktypes, desc = "Networks", leave = False):
            if debug: print(placeid + ": Analyzing results: " + networktype)
            try:
                metrics, cov = calculate_metrics(Gs[networktype], Gs[networktype + "_simplified"], Gs['carall'], nnids, empty_metrics, buffer_walk, numnodepairs, debug)
                for key, val in metrics.items():
                    output_place[networktype][key] = val
                covs[networktype] = cov
            except:
                print(networktype + " is empty")
        # Save the covers
        write_result(covs, "pickle", placeid, "", "", "existing_covers.pickle")
        
        # Write to CSV
        write_result(output_place, "dictnested", placeid, "", "", "existing.csv", empty_metrics)
    else:
        print(placeid + ": Existing infrastructure has already been analyzed")

## Analyze POI based results

In [ ]:
for placeid, placeinfo in tqdm(cities.items(), desc = "Cities"):
    print(placeid + ": Analyzing results")

    # Load networks
    G_carall = csv_to_ig(PATH["data"] + placeid + "/", placeid, 'carall')
    Gexisting = {}
    for networktype in ["biketrack", "bikeable"]:
        try:
            Gexisting[networktype] = csv_to_ig(PATH["data"] + placeid + "/", placeid, networktype)
        except:
            print(networktype + " is empty")
        
    
    # Load POIs
    with open(PATH["data"] + placeid + "/" + placeid + '_poi_' + poi_source + '_nnidscarall.csv') as f:
        nnids = [int(line.rstrip()) for line in f]
            
    # Load results
    filename = placeid + '_poi_' + poi_source + "_" + prune_measure
    resultfile = open(PATH["results"] + placeid + "/" + filename + ".pickle",'rb')
    res = pickle.load(resultfile)
    resultfile.close()
    if debug: pp.pprint(res)
         
    # Calculate
    # output contains lists for all the prune_quantile values of the corresponding results
    output, covs = calculate_metrics_additively(res["GTs"], res["GT_abstracts"], res["prune_quantiles"], G_carall, nnids, buffer_walk, numnodepairs, debug, True, Gexisting)
    output_MST, cov_MST = calculate_metrics(res["MST"], res["MST_abstract"], G_carall, nnids, output, buffer_walk, numnodepairs, debug, True, ig.Graph(), Polygon(), False, Gexisting)
        
    # Save the covers
    write_result(covs, "pickle", placeid, poi_source, prune_measure, "_covers.pickle")
#     write_result(covs_carminusbike, "pickle", placeid, poi_source, prune_measure, "_covers_carminusbike.pickle")
    write_result(cov_MST, "pickle", placeid, poi_source, prune_measure, "_cover_mst.pickle")
        
    # Write to CSV
    write_result(output, "dict", placeid, poi_source, prune_measure, ".csv")
#     write_result(output_carminusbike, "dict", placeid, poi_source, prune_measure, "_carminusbike.csv")
#     write_result(output_carconstrictedbike, "dict", placeid, poi_source, prune_measure, "_carconstrictedbike.csv")
    write_result(output_MST, "dict", placeid, poi_source, "", "mst.csv")

In [ ]:
Audio(sound_file, autoplay=True) 